In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import numpy as np

In [ ]:
!pip install transformers[torch]
!pip install accelerate

In [ ]:
!pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

data = load_dataset("kde4", lang1 = "en", lang2 = "vi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-vi")

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-vi")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
split_data = data['train'].train_test_split(train_size = 0.8, seed = 13)
split_data['validation'] = split_data.pop("test")

train_dict = split_data['train']
val_dict = split_data['validation']

train_dict[:5]['translation']

[{'en': 'Laser', 'vi': 'Laser'},
 {'en': 'Export Actions', 'vi': 'Hành động'},
 {'en': 'Layer', 'vi': '& Lớp'},
 {'en': 'Load', 'vi': 'Tải'},
 {'en': 'Channel', 'vi': 'Kênh'}]

In [ ]:
max_seq_length = 128

def preprocess(examples):
  input_tokens = [ex['en'] for ex in examples['translation']]
  output_tokens = [ex['vi'] for ex in examples['translation']]

  model_inputs = tokenizer(
      input_tokens, text_target = output_tokens, max_length = max_seq_length, truncation = True
  )

  return model_inputs

s = preprocess(train_dict[:2])
print(tokenizer.convert_ids_to_tokens(s['input_ids'][1]))
print(tokenizer.convert_ids_to_tokens(s['labels'][1]))
print(train_dict[1]['translation'])
preprocess(train_dict[:2])

['▁Export', '▁Action', 's', '</s>']
['▁Hành', '▁động', '</s>']
{'en': 'Export Actions', 'vi': 'Hành động'}


{'input_ids': [[27425, 0], [15463, 11450, 11, 0]], 'attention_mask': [[1, 1], [1, 1, 1, 1]], 'labels': [[27425, 0], [5023, 307, 0]]}

In [ ]:
tokenized_datasets = split_data.map(
    preprocess,
    batched=True,
    remove_columns=split_data["train"].column_names,
)

Map:   0%|          | 0/34225 [00:00<?, ? examples/s]

Map:   0%|          | 0/8557 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch = data_collator([tokenized_datasets['train'][i] for i in range(0, 3)])
batch

{'input_ids': tensor([[27425,     0, 53684, 53684],
        [15463, 11450,    11,     0],
        [16325,     0, 53684, 53684]]), 'attention_mask': tensor([[1, 1, 0, 0],
        [1, 1, 1, 1],
        [1, 1, 0, 0]]), 'labels': tensor([[27425,     0,  -100],
        [ 5023,   307,     0],
        [  942,  9785,     0]]), 'decoder_input_ids': tensor([[53684, 27425,     0],
        [53684,  5023,   307],
        [53684,   942,  9785]])}

In [ ]:
print(tokenizer.batch_decode(batch['decoder_input_ids'], skip_special_tokens = True))

['Laser', 'Hành động', '& Lớp']


In [ ]:
# input, output =

input_ids = batch['input_ids']
labels = batch['labels']
decoder_input_ids = batch['decoder_input_ids']

print(tokenizer.convert_ids_to_tokens(input_ids[1]))
print(tokenizer.convert_ids_to_tokens(labels[1]))
print(tokenizer.convert_ids_to_tokens(decoder_input_ids[1]))
batch.keys()

['▁Export', '▁Action', 's', '</s>']
['▁Hành', '▁động', '</s>']
['<pad>', '▁Hành', '▁động']


dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
predictions = [
    "This plugin translates web pages."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)


{'score': 7.559185144683521,
 'counts': [5, 2, 0, 0],
 'totals': [6, 5, 4, 3],
 'precisions': [83.33333333333333, 40.0, 12.5, 8.333333333333334],
 'bp': 0.31140322391459774,
 'sys_len': 6,
 'ref_len': 13}

In [ ]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds

  decode_preds = tokenizer.batch_decode(preds, skip_special_tokens = True)

  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

  decode_preds = [decode_pred.strip() for decode_pred in decode_preds]
  decode_labels = [[decode_label.strip()] for decode_label in decode_labels]
  score = metric.compute(predictions = decode_preds, references = decode_labels)

  return {'bleu score': score}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(tokenized_datasets['train'],
                          shuffle = True,
                          batch_size = 8,
                          collate_fn = data_collator)

eval_loader = DataLoader(tokenized_datasets['validation'],
                         shuffle = True,
                         batch_size = 8,
                         collate_fn = data_collator)

In [ ]:
# for batch in train_loader:
#   print(model(**batch)['loss'])
#   break

model.train()

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(53685, 512, padding_idx=53684)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(53685, 512, padding_idx=53684)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from accelerate import Accelerator

num_epochs = 2
num_training_steps = len(train_loader)
optimizer = AdamW(model.parameters(), lr = 2e-5, weight_decay=0.01)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

saved_loss = {'train': [], 'eval': []}

for epoch in range(num_epochs):
  avg_train_loss, avg_eval_loss = 0, 0

  model.train()
  for batch in train_loader:
    outputs = model(**batch)
    loss = outputs['loss']
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    avg_train_loss += loss

  avg_train_loss /= len(train_loader)
  print("Epoch {}, avg train loss {}\n".format(epoch, avg_train_loss))
  saved_loss['train'].append(avg_train_loss)

  model.eval()
  for batch in eval_loader:
    with torch.no_grad():
      outputs = model(**batch)
      loss = outputs['loss']
      avg_eval_loss += loss
  avg_eval_loss /= len(eval_loader)
  print("Epoch {}, avg eval loss {}\n".format(epoch, avg_eval_loss))
  print("--------------\n")
  saved_loss['eval'].append(avg_eval_loss)

